Datasets help in collecting data from production, staging, evaluations and manually

In [1]:
import phoenix as px
# px.launch_app()

/media/uberdev/ddrv/telemetenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
from dotenv import load_dotenv
load_dotenv("/media/uberdev/ddrv/gitFolders/python_de_learners_data/.env")

True

In [3]:
# simple data point

sdf = pd.DataFrame(
    [{
        "question": "What is paul graham famous for",
        "answer": "Co-founding Y-combinator and writing on startups",
        "metadata": ["topic", "tech"]
    }]
)

In [6]:
phoenix_client = px.Client(endpoint="http://localhost:6006/")

INFO:httpx:HTTP Request: GET http://localhost:6006/arize_phoenix_version "HTTP/1.1 200 OK"


In [7]:
dataset = phoenix_client.upload_dataset(
    dataframe=sdf,
    dataset_name="sample-dataset",
    input_keys=["question"],
    output_keys=["answer"],
    metadata_keys=["metadata"]
)

INFO:httpx:HTTP Request: POST http://localhost:6006/v1/datasets/upload?sync=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6006/v1/datasets/RGF0YXNldDox/examples "HTTP/1.1 200 OK"


📤 Uploading dataset...
💾 Examples uploaded: http://localhost:6006/datasets/RGF0YXNldDox/examples
🗄️ Dataset version ID: RGF0YXNldFZlcnNpb246MQ==


In [8]:
from openai import OpenAI
from phoenix.experiments.types import Example
# above import gets the datatypes under Experiments

In [9]:
import os
openai_client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [10]:
task_prompt = "Answer in few words: {question}"

In [11]:
def task(example: Example) -> str:
    question = example.input["question"]
    messages_context = task_prompt.format(question=question)
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini", 
        messages= [
                {
                    "role": "user",
                    "content": messages_context
                }
        ]
    )
    return response.choices[0].message.content

In [12]:
# we can look at couple of evaluators built in to phoenix
from phoenix.experiments.evaluators import (
    ContainsAllKeywords,
    ConcisenessEvaluator,
)
# pull the model classes that are build for evaluation
from phoenix.evals.models import OpenAIModel
contains_kw = ContainsAllKeywords(keywords=["Y Combinator", "YC"])
model = OpenAIModel(model="gpt-4o-mini")
conciseness = ConcisenessEvaluator(model=model)

In [13]:
# define custom similarity with Code
from typing import Any, Dict

def jaccard_similarity(output: str, expected: Dict[str, Any]) -> float:
    actual_words = set(output.lower().split(" "))
    expected_words = set(expected["answer"].lower().split(" "))
    words_in_common = actual_words.intersection(expected_words)
    all_words = actual_words.union(expected_words)
    # https://en.wikipedia.org/wiki/Jaccard_index
    return len(words_in_common) / len(all_words)
    

In [14]:
from phoenix.experiments.evaluators import create_evaluator
from typing import Any, Dict

In [15]:
eval_prompt_template = """
Given the QUESTION and REFERENCE_ANSWER, determine whether the ANSWER is accurate.
Output only a single word (accurate or inaccurate).

QUESTION: {question}

REFERENCE_ANSWER: {reference_answer}

ANSWER: {answer}

ACCURACY (accurate / inaccurate):
"""

In [16]:
@create_evaluator(kind="llm")  # need the decorator or the kind will default to "code"
def accuracy(input: Dict[str, Any], 
             output: str, 
             expected: Dict[str, Any]) -> float:
    # message contents are manually built
    message_content = eval_prompt_template.format(
        question=input["question"],
        reference_answer=expected["answer"], 
        answer=output
    )
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", 
                   "content": message_content}]
    )
    response_message_content = response.choices[0].message.content.lower().strip()
    return 1.0 if response_message_content == "accurate" else 0.0

In [17]:
# run experiment, which requires dataset, task, evaluators
from phoenix.experiments import run_experiment
experiment = run_experiment(
    dataset=dataset, # phoenix object
    task=task,
    experiment_name="sample-expt",
    evaluators=[jaccard_similarity, accuracy],
)

INFO:httpx:HTTP Request: POST http://127.0.0.1:6006/v1/datasets/RGF0YXNldDox/experiments "HTTP/1.1 200 OK"


🧪 Experiment started.
📺 View dataset experiments: http://127.0.0.1:6006/datasets/RGF0YXNldDox/experiments
🔗 View this experiment: http://127.0.0.1:6006/datasets/RGF0YXNldDox/compare?experimentId=RXhwZXJpbWVudDox


running tasks |          | 0/1 (0.0%) | ⏳ 00:00<? | ?it/sINFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:6006/v1/experiments/RXhwZXJpbWVudDox/runs "HTTP/1.1 200 OK"
running tasks |██████████| 1/1 (100.0%) | ⏳ 00:01<00:00 |  1.43s/it


✅ Task runs completed.
🧠 Evaluation started.


running experiment evaluations |          | 0/2 (0.0%) | ⏳ 00:00<? | ?it/sINFO:httpx:HTTP Request: POST http://127.0.0.1:6006/v1/experiment_evaluations "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:6006/v1/experiment_evaluations "HTTP/1.1 200 OK"
running experiment evaluations |██████████| 2/2 (100.0%) | ⏳ 00:02<00:00 |  1.14s/it



🔗 View this experiment: http://127.0.0.1:6006/datasets/RGF0YXNldDox/compare?experimentId=RXhwZXJpbWVudDox

Experiment Summary (08/16/24 09:02 PM +0530)
--------------------------------------------
            evaluator  n  n_scores  avg_score
0            accuracy  1         1       1.00
1  jaccard_similarity  1         1       0.25

Tasks Summary (08/16/24 09:02 PM +0530)
---------------------------------------
   n_examples  n_runs  n_errors
0           1       1         0


In [18]:
px.close_app()